In [1]:
import mlflow
from pyspark.sql import SparkSession

# 1. Spark se conecta localmente
# Usamos 'local[*]' para usar todos los cores disponibles en el contenedor
spark = (
    SparkSession.builder
    .appName("Test Integrado")
    .master("local[*]")
    .config("spark.driver.memory", "16g")
    .getOrCreate()
)

print(f"Versión de Spark: {spark.version}")
print(f"Versión de MLflow: {mlflow.__version__}")

# 2. MLflow se conecta a su servidor en localhost (dentro del contenedor)
mlflow.set_tracking_uri("http://localhost:5000")

# 3. Ejecutar un experimento de MLflow
mlflow.set_experiment("Experimento_Todo_En_Uno")

with mlflow.start_run(run_name="Run de Prueba"):
    mlflow.log_param("entorno", "Contenedor Único")
    
    print("Ejecutando un trabajo simple en Spark...")
    data = [("Python", 10), ("Spark", 20), ("MLflow", 30)]
    df = spark.createDataFrame(data, ["componente", "valor"])
    
    df.show()
    
    avg_value = df.toPandas()['valor'].mean()
    mlflow.log_metric("valor_promedio", avg_value)
    
    print("Experimento y run de MLflow registrados con éxito.")

spark.stop()
 

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/12/12 15:19:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/12/12 15:19:12 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/12/12 15:19:12 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


Versión de Spark: 3.5.1
Versión de MLflow: 2.11.1


2025/12/12 15:19:13 INFO mlflow.tracking.fluent: Experiment with name 'Experimento_Todo_En_Uno' does not exist. Creating a new experiment.
2025/12/12 15:19:13 WARNING mlflow.utils.git_utils: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Bad git executable.
The git executable must be specified in one of the following ways:
    - be included in your $PATH
    - be set via $GIT_PYTHON_GIT_EXECUTABLE
    - explicitly set via git.refresh(<full-path-to-git-executable>)

All git commands will error until this is rectified.

This initial message can be silenced or aggravated in the future by setting the
$GIT_PYTHON_REFRESH environment variable. Use one of the following values:
    - quiet|q|silence|s|silent|none|n|0: for no message or exception
    - warn|w|warning|log|l|1: for a warning message (logging level CRITICAL, displayed by default)
    - error|e|exception|raise|r|2: for a raised exception

Example:
  

Ejecutando un trabajo simple en Spark...


+----------+-----+
|componente|valor|
+----------+-----+
|    Python|   10|
|     Spark|   20|
|    MLflow|   30|
+----------+-----+

Experimento y run de MLflow registrados con éxito.


## Configuración del Entorno de Spark
El primer paso consiste en inicializar una SparkSession, que es el punto de entrada para programar con la API de Spark. El siguiente script también automatiza la descarga del conjunto de datos.


In [2]:
import os
import urllib.request
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

# 1. Inicialización de la SparkSession
# El master 'local[*]' instruye a Spark para utilizar todos los núcleos de CPU
# disponibles en la máquina local, simulando un entorno de trabajo paralelo.
spark = (
    SparkSession.builder
    .appName("TutorialComputacionDistribuida")
    .master("local[*]")
    .getOrCreate()
)

# 2. Descarga y gestión del conjunto de datos
data_dir = "data"
filename = "yellow_tripdata_2024-01.parquet"
filepath = os.path.join(data_dir, filename)

if not os.path.exists(filepath):
    print(f"Iniciando descarga del conjunto de datos: {filename}")
    os.makedirs(data_dir, exist_ok=True)
    url = f"https://d37ci6vzurychx.cloudfront.net/trip-data/{filename}"
    urllib.request.urlretrieve(url, filepath)
    print("Descarga finalizada.")
else:
    print("El conjunto de datos ya se encuentra en el directorio local.")

# 3. Verificación del contexto de Spark
print(f"SparkSession iniciada. Versión: {spark.version}")
spark


25/12/12 15:19:22 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/12/12 15:19:22 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


Iniciando descarga del conjunto de datos: yellow_tripdata_2024-01.parquet
Descarga finalizada.
SparkSession iniciada. Versión: 3.5.1


## Ejercicio 1: El Cuello de Botella del Mononodo con Pandas
Este ejercicio demuestra las limitaciones inherentes al procesamiento en un solo nodo. Se utilizará la librería Pandas para cargar y procesar el dataset.


In [3]:
import pandas as pd

print("Cargando el conjunto de datos en un DataFrame de Pandas...")

# Medición del tiempo de carga y consumo de memoria
%time df_pandas = pd.read_parquet(filepath)

# Medición del tiempo de una operación de agregación simple
%time conteo_pandas = df_pandas['passenger_count'].value_counts()

print("\n--- Análisis de Rendimiento con Pandas ---")
print(f"Consumo de memoria RAM: {df_pandas.memory_usage(deep=True).sum() / 1e9:.2f} GB")
print("\nResultado del conteo de pasajeros:")
print(conteo_pandas)


Cargando el conjunto de datos en un DataFrame de Pandas...
CPU times: user 666 ms, sys: 5.7 s, total: 6.37 s
Wall time: 1.07 s
CPU times: user 17.7 ms, sys: 3.2 ms, total: 20.9 ms
Wall time: 19.1 ms

--- Análisis de Rendimiento con Pandas ---
Consumo de memoria RAM: 0.56 GB

Resultado del conteo de pasajeros:
passenger_count
1.0    2188739
2.0     405103
3.0      91262
4.0      51974
5.0      33506
0.0      31465
6.0      22353
8.0         51
7.0          8
9.0          1
Name: count, dtype: int64


## Ejercicio 2: Ingesta de Datos Distribuida y Evaluación Perezosa
A continuación, se realiza la misma operación de carga utilizando Spark para ilustrar un concepto fundamental: la evaluación perezosa (lazy evaluation).

In [4]:
print("Cargando el conjunto de datos en un DataFrame de Spark...")

# La lectura del archivo es una transformación perezosa. Es instantánea.
%time df_spark = spark.read.parquet(filepath)

# .count() es una acción, la cual dispara la ejecución del trabajo.
%time total_filas = df_spark.count()

print(f"\nEl DataFrame de Spark contiene {total_filas} registros.")

Cargando el conjunto de datos en un DataFrame de Spark...
CPU times: user 7.37 ms, sys: 123 ms, total: 131 ms
Wall time: 516 ms
CPU times: user 3.85 ms, sys: 58.9 ms, total: 62.7 ms
Wall time: 615 ms

El DataFrame de Spark contiene 2964624 registros.


## Ejercicio 3: Paralelismo y Particiones
El paralelismo en Spark se logra dividiendo los datos en particiones. Un DataFrame es una abstracción sobre un RDD (Resilient Distributed Dataset), el cual es una colección de elementos particionados. Cada partición puede ser procesada de forma independiente y en paralelo por un executor en un nodo del clúster.

In [5]:
# 1. Obtener el número de particiones por defecto
num_particiones = df_spark.rdd.getNumPartitions()
print(f"El DataFrame ha sido dividido en {num_particiones} particiones.")

# 2. Inspeccionar la distribución de registros por partición
# Se utiliza el RDD subyacente para aplicar una función a cada partición.
filas_por_particion = df_spark.rdd.mapPartitions(lambda iter: [sum(1 for _ in iter)]).collect()

print(f"\nDistribución de registros en {len(filas_por_particion)} particiones:")
print(filas_por_particion)

El DataFrame ha sido dividido en 12 particiones.


25/12/12 15:19:34 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors
25/12/12 15:19:34 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors
[Stage 4:===============================================>         (10 + 2) / 12]


Distribución de registros en 12 particiones:
[0, 1048576, 0, 0, 0, 1048576, 0, 0, 0, 867472, 0, 0]


## Ejercicio 4: Modelo de Ejecución - Transformaciones y Acciones
El modelo de programación de Spark se basa en la construcción de un plan de ejecución a través de transformaciones, que culmina con la ejecución de dicho plan mediante una acción.


In [6]:
# --- 1. Definición del Plan Lógico (Transformaciones) ---
# Estas operaciones son perezosas y construyen un Grafo Acíclico Dirigido (DAG).

# Filtro de viajes con más de 1 pasajero y distancia superior a 5 millas
df_filtrado = df_spark.filter(
    (F.col("passenger_count") > 1) &
    (F.col("trip_distance") > 5)
)

# Creación de una nueva columna con el porcentaje de propina
df_con_propina_pct = df_filtrado.withColumn(
    "propina_pct",
    (F.col("tip_amount") / F.col("total_amount")) * 100
)

# Agregación para calcular la propina promedio por número de pasajeros
df_agrupado = df_con_propina_pct.groupBy("passenger_count").agg(
    F.avg("propina_pct").alias("propina_promedio_pct"),
    F.count("*").alias("conteo_viajes")
)

# Ordenamiento del resultado
df_plan_final = df_agrupado.orderBy(F.col("passenger_count").desc())

# --- 2. Inspección del Plan Físico de Ejecución ---
print("Plan de ejecución optimizado por Spark (DAG):")
df_plan_final.explain()




Plan de ejecución optimizado por Spark (DAG):
== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Sort [passenger_count#16L DESC NULLS LAST], true, 0
   +- Exchange rangepartitioning(passenger_count#16L DESC NULLS LAST, 200), ENSURE_REQUIREMENTS, [plan_id=80]
      +- HashAggregate(keys=[passenger_count#16L], functions=[avg(propina_pct#76), count(1)])
         +- Exchange hashpartitioning(passenger_count#16L, 200), ENSURE_REQUIREMENTS, [plan_id=77]
            +- HashAggregate(keys=[passenger_count#16L], functions=[partial_avg(propina_pct#76), partial_count(1)])
               +- Project [passenger_count#16L, ((tip_amount#26 / total_amount#29) * 100.0) AS propina_pct#76]
                  +- Filter (((isnotnull(passenger_count#16L) AND isnotnull(trip_distance#17)) AND (passenger_count#16L > 1)) AND (trip_distance#17 > 5.0))
                     +- FileScan parquet [passenger_count#16L,trip_distance#17,tip_amount#26,total_amount#29] Batched: true, DataFilters: [isnotnull(passenge

## Ejercicio 5: Ejecución del Plan y la Operación de Shuffle
Finalmente, una acción como .collect() o .show() desencadena la ejecución del DAG.


In [7]:
# --- 3. Ejecución del Plan (Acción) ---
# La acción .collect() transfiere los resultados al nodo driver.
# ADVERTENCIA: Usar .collect() solo con resultados de tamaño manejable.
print("\n--- Ejecutando el plan de computación distribuida... ---")

%time resultados = df_plan_final.collect()

print("\n--- Resultados del Análisis ---")
for fila in resultados:
    print(f"Pasajeros: {fila['passenger_count']}, \tConteo: {fila['conteo_viajes']}, \tPropina Promedio: {fila['propina_promedio_pct']:.2f}%")



--- Ejecutando el plan de computación distribuida... ---
CPU times: user 1.16 ms, sys: 7.93 ms, total: 9.09 ms
Wall time: 1.06 s

--- Resultados del Análisis ---
Pasajeros: 8, 	Conteo: 5, 	Propina Promedio: 11.98%
Pasajeros: 7, 	Conteo: 2, 	Propina Promedio: 18.32%
Pasajeros: 6, 	Conteo: 2917, 	Propina Promedio: 11.75%
Pasajeros: 5, 	Conteo: 4624, 	Propina Promedio: 11.64%
Pasajeros: 4, 	Conteo: 10183, 	Propina Promedio: 9.57%
Pasajeros: 3, 	Conteo: 16344, 	Propina Promedio: 10.46%
Pasajeros: 2, 	Conteo: 74759, 	Propina Promedio: 11.18%


## Clase 2

Continuamos estudiando Spark. Observa atentamente los tiempos de ejecución.

In [8]:
print("Cargando el DataFrame en Spark...")
print(filepath)
%time df_spark = spark.read.parquet(filepath)

print("\n--- Ejecutando una 'Acción' (como .count()) ---")
%time total_filas = df_spark.count()

print(f"El DataFrame de Spark contiene {total_filas} registros.")

Cargando el DataFrame en Spark...
data/yellow_tripdata_2024-01.parquet
CPU times: user 959 μs, sys: 2.82 ms, total: 3.78 ms
Wall time: 109 ms

--- Ejecutando una 'Acción' (como .count()) ---
CPU times: user 314 μs, sys: 925 μs, total: 1.24 ms
Wall time: 240 ms
El DataFrame de Spark contiene 2964624 registros.


**Análisis y Filosofía de Spark:**

* `spark.read.parquet(filepath)`: ¡El tiempo de ejecución fue casi **cero** (milisegundos)!
* **¿Por Qué?** Porque Spark no cargó los datos. Esto es la **Evaluación Perezosa (Lazy Evaluation)**.
* **Transformaciones (La Receta):** `spark.read.parquet` es una **Transformación**. Es una instrucción, una "promesa" de que *eventualmente* leerás ese archivo. Spark simplemente anota en su plan: "OK, empezaré por este archivo".
* **Acciones (La Orden):** `.count()` es una **Acción**. Es la orden de "¡Ejecuta el plan y dame un resultado!". Solo cuando pides una acción, Spark realmente empieza a trabajar.
* **La Ventaja:** Esto permite a Spark construir un plan de ejecución complejo y optimizarlo *antes* de mover un solo byte de datos.

## Ejercicio ¿Cómo procesa Spark?
¿Cómo procesa Spark los datos sin cargarlos todos a la RAM? Dividiéndolos en **Particiones**. Una partición es un "trozo" del conjunto de datos que puede ser procesado por un trabajador (un núcleo de CPU).

In [9]:
# 1. ¿Cuántas particiones (lotes) creó Spark?
num_particiones = df_spark.rdd.getNumPartitions()
print(f"El DataFrame ha sido dividido en {num_particiones} particiones.")

# 2. Miremos DENTRO de las particiones (¿cuántas filas tiene cada una?)
filas_por_particion = df_spark.rdd.mapPartitions(lambda iter: [sum(1 for _ in iter)]).collect()
print(f"\nDistribución de registros en {len(filas_por_particion)} particiones:")
print(filas_por_particion)

El DataFrame ha sido dividido en 12 particiones.


[Stage 17:==============================================>         (10 + 2) / 12]


Distribución de registros en 12 particiones:
[0, 1048576, 0, 0, 0, 1048576, 0, 0, 0, 867472, 0, 0]


**Salida Esperada (¡Nuestra Primera Pista!):**

El DataFrame ha sido dividido en 12 particiones. Distribución de registros en 12 particiones: [0, 0, 1048576, 0, 0, 0, 1048576, 0, 0, 0, 867472, 0]


**Análisis y Filosofía de Spark:**

* `df_spark.rdd.getNumPartitions()`: `rdd` es la API de bajo nivel de Spark. Le estamos preguntando: "¿En cuántos lotes se dividió el trabajo?".
* **Grado de Paralelismo:** Responde 12. Esto es porque nuestro `.master("local[*]")` detectó 12 núcleos de CPU, por lo que 12 es el número de hilos "trabajadores" por defecto.
* **Desbalanceo de Datos (Data Skew):** ¡Este es el punto clave! La lista `[0, 0, 1048576, ...]` nos dice que **9 de nuestros 12 trabajadores no están haciendo nada**. Todo el trabajo de lectura se concentra en solo 3 particiones (trabajadores).
* **¿Por qué?** No es un error. Es un artefacto de la lectura. El archivo Parquet que descargamos está almacenado en el disco en 3 "Row Groups" (bloques) principales. Spark, al leerlo, asignó cada bloque a una partición.
* **Conclusión:** Aunque tenemos 12 núcleos listos, en la primera etapa del trabajo, solo 3 están activos. **Lo arreglaremos en lo que sigue.**

**Análisis y Filosofía de Spark:**

* `df_spark.rdd.getNumPartitions()`: `rdd` es la API de bajo nivel de Spark. Le estamos preguntando: "¿En cuántos lotes se dividió el trabajo?".
* **Grado de Paralelismo:** Responde 12. Esto es porque nuestro `.master("local[*]")` detectó 12 núcleos de CPU, por lo que 12 es el número de hilos "trabajadores" por defecto.
* **Desbalanceo de Datos (Data Skew):** ¡Este es el punto clave! La lista `[0, 0, 1048576, ...]` nos dice que **9 de nuestros 12 trabajadores no están haciendo nada**. Todo el trabajo de lectura se concentra en solo 3 particiones (trabajadores).
* **¿Por qué?** No es un error. Es un artefacto de la lectura. El archivo Parquet que descargamos está almacenado en el disco en 3 "Row Groups" (bloques) principales. Spark, al leerlo, asignó cada bloque a una partición.
* **Conclusión:** Aunque tenemos 12 núcleos listos, en la primera etapa del trabajo, solo 3 están activos. **Lo arreglaremos en el Ejercicio 6.**

## Ejercicio: Optimizando - Uso Completo de Recursos

Solo 3 de nuestros 12 núcleos estaban trabajando en la lectura. Vamos a arreglar esto.

**La Solución:** `repartition()`
Le daremos a Spark una orden explícita para que baraje los datos *inmediatamente* después de leerlos y los redistribuya equitativamente en 12 particiones.

In [10]:
# 1. Creamos un nuevo DataFrame, forzando la redistribución
print(f"Particiones originales: {df_spark.rdd.getNumPartitions()}")
df_reparticionado = df_spark.repartition(12)

# 2. Verifiquemos la nueva distribución
# .cache() es una transformación que le dice a Spark "guarda el resultado de
# esta repartición en memoria para que no tengamos que hacerla de nuevo".
df_reparticionado.cache()
print(f"Nuevas particiones: {df_reparticionado.rdd.getNumPartitions()}")
print("\nVerificando la nueva distribución (esto tomará un momento)...")

# Esta acción (.count()) fuerza la ejecución del .repartition() y .cache()
df_reparticionado.count() 

# Ahora miremos dentro de las particiones cacheadas
nuevas_filas_por_particion = df_reparticionado.rdd.mapPartitions(lambda iter: [sum(1 for _ in iter)]).collect()
print(f"Distribución de registros en 12 particiones (después de repartition):")
print(nuevas_filas_por_particion)

# Liberar la memoria cacheada
df_reparticionado.unpersist()

Particiones originales: 12


[Stage 20:=======================>                                 (5 + 7) / 12]

Nuevas particiones: 12

Verificando la nueva distribución (esto tomará un momento)...


[Stage 27:============================>                            (6 + 6) / 12]

Distribución de registros en 12 particiones (después de repartition):
[247052, 247051, 247052, 247052, 247053, 247053, 247052, 247052, 247051, 247052, 247052, 247052]


DataFrame[VendorID: int, tpep_pickup_datetime: timestamp_ntz, tpep_dropoff_datetime: timestamp_ntz, passenger_count: bigint, trip_distance: double, RatecodeID: bigint, store_and_fwd_flag: string, PULocationID: int, DOLocationID: int, payment_type: bigint, fare_amount: double, extra: double, mta_tax: double, tip_amount: double, tolls_amount: double, improvement_surcharge: double, total_amount: double, congestion_surcharge: double, Airport_fee: double]

**Análisis y Filosofía de Spark:**

* `df_spark.repartition(12)`: Es una **Transformación Ancha (Wide)**. Le dimos a Spark la orden explícita de "ejecutar un SHUFFLE completo y redistribuir todos los datos en 12 nuevas particiones de tamaño (casi) idéntico".
* **¡Éxito!** La nueva distribución de filas es perfectamente balanceada.
* **La Ventaja:** Ahora, cualquier operación que hagamos (como nuestro `filter` o `project`) utilizará **los 12 núcleos en paralelo** desde el primer segundo.
* **El Costo:** `repartition()` es en sí mismo un *shuffle* completo, ¡lo cual es costoso! Introdujimos un costo inicial, pero lo hicimos para que el resto del procesamiento sea mucho más rápido y eficiente.

## Sintáxis de spark

### Parte 1: Selección y Filtro (El `SELECT` y `WHERE` de SQL)

Empecemos por lo básico: cómo seleccionar columnas y filtrar filas.

In [11]:
# Siempre importamos las funciones de SQL
import pyspark.sql.functions as F

# Carguemos nuestro DataFrame original (el desbalanceado está bien para esto)
df_spark = spark.read.parquet("data/yellow_tripdata_2024-01.parquet")

La función más básica. Le dices qué columnas quieres.

In [13]:
# SQL: SELECT passenger_count, trip_distance FROM ...
df_seleccion = df_spark.select("passenger_count", "trip_distance", "total_amount")

print("DataFrame solo con 3 columnas:")
df_seleccion.show(5)

DataFrame solo con 3 columnas:
+---------------+-------------+------------+
|passenger_count|trip_distance|total_amount|
+---------------+-------------+------------+
|              1|         1.72|        22.7|
|              1|          1.8|       18.75|
|              1|          4.7|        31.3|
|              1|          1.4|        17.0|
|              1|          0.8|        16.1|
+---------------+-------------+------------+
only showing top 5 rows



`selectExpr` (Select Expression) te permite escribir pseudo-SQL dentro de una string de Python. Es muy útil para cálculos rápidos o para renombrar.

In [14]:
# SQL: SELECT trip_distance, trip_distance * 1.60934 AS trip_distance_km FROM ...
df_expr = df_spark.selectExpr(
    "trip_distance", 
    "trip_distance * 1.60934 AS trip_distance_km",
    "passenger_count AS pasajeros"
)

print("DataFrame con columnas calculadas y renombradas:")
df_expr.show(5)

DataFrame con columnas calculadas y renombradas:
+-------------+------------------+---------+
|trip_distance|  trip_distance_km|pasajeros|
+-------------+------------------+---------+
|         1.72|         2.7680648|        1|
|          1.8|          2.896812|        1|
|          4.7|          7.563898|        1|
|          1.4|2.2530759999999996|        1|
|          0.8|1.2874720000000002|        1|
+-------------+------------------+---------+
only showing top 5 rows



**`filter()` y `F.col()` - El `WHERE` de SQL**

Aquí es donde entra la filosofía de `F.col()`.

* `"passenger_count"` (string): Es solo el *nombre* de la columna.
* `F.col("passenger_count")` (objeto Columna): Es la *columna real* sobre la que podemos hacer operaciones (como `>`, `==`, `+`, etc.).

In [15]:
# SQL: WHERE passenger_count > 1 AND trip_distance > 5
df_filtrado = df_spark.filter(
    (F.col("passenger_count") > 1) & 
    (F.col("trip_distance") > 5)
)

print("Viajes largos con más de 1 pasajero:")
df_filtrado.show(5)

# Recordatorio de sintaxis:
# En PySpark, usa '&' (AND), '|' (OR), y '~' (NOT)

Viajes largos con más de 1 pasajero:
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       2| 2024-01-01 00:49:44|  2024-01-01 01:15:47|              2|        10.82|         1|                 N|         138|         181|           

**`where()` - El alias de `filter()`**

`where()` hace *exactamente* lo mismo que `filter()`. Es solo un alias para que los que vienen de SQL se sientan más cómodos.

In [16]:
# SQL: WHERE payment_type = 1
df_where = df_spark.where(F.col("payment_type") == 1)

print("Viajes pagados con Tarjeta de Crédito (payment_type 1):")
df_where.show(5)

Viajes pagados con Tarjeta de Crédito (payment_type 1):
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       1| 2024-01-01 00:03:00|  2024-01-01 00:09:36|              1|          1.8|         1|                 N|         140|     

**`like()` - Búsqueda de patrones**

Al igual que en SQL, `like()` busca patrones en strings. `store_and_fwd_flag` es una columna con 'Y' o 'N'.

In [17]:
# SQL: WHERE store_and_fwd_flag LIKE 'Y'
df_like = df_spark.filter( F.col("store_and_fwd_flag").like("Y") )

print(f"Viajes con 'store_and_fwd_flag' = Y:")
df_like.show(10)

Viajes con 'store_and_fwd_flag' = Y:
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       1| 2024-01-01 00:51:58|  2024-01-01 01:05:44|              2|          2.9|         1|                 Y|         264|         264|           

**`isin()` - Múltiples `OR`**

`isin()` es un atajo para `WHERE mi_columna = 'A' OR mi_columna = 'B' ...`

In [18]:
# Busquemos viajes pagados con "Tarjeta" (1) o "Efectivo" (2)
df_isin = df_spark.filter( F.col("payment_type").isin(1, 2) )

print("Viajes pagados con Tarjeta o Efectivo:")
df_isin.show(5)

Viajes pagados con Tarjeta o Efectivo:
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       2| 2024-01-01 00:57:55|  2024-01-01 01:17:43|              1|         1.72|         1|                 N|         186|          79|         

### Parte 2: Manipulación de Columnas (Ingeniería de Características)

Aquí es donde Spark brilla. Crear nuevas columnas es la base de la ingeniería de características (feature engineering).

**`withColumn()` - La función más importante**

`withColumn("nombre_columna", <operación>)` es la forma estándar de crear una nueva columna o reemplazar una existente.

In [19]:
# Vamos a crear la columna 'propina_pct' que usamos en la clase anterior
df_con_columna = df_spark.withColumn(
    "propina_pct",
    (F.col("tip_amount") / F.col("total_amount")) * 100
)

print("DataFrame con 'propina_pct' (puede ser NaN si total_amount es 0):")
df_con_columna.select("VendorID","tip_amount", "total_amount", "propina_pct","passenger_count").show(5)

DataFrame con 'propina_pct' (puede ser NaN si total_amount es 0):
+--------+----------+------------+------------------+---------------+
|VendorID|tip_amount|total_amount|       propina_pct|passenger_count|
+--------+----------+------------+------------------+---------------+
|       2|       0.0|        22.7|               0.0|              1|
|       1|      3.75|       18.75|              20.0|              1|
|       1|       3.0|        31.3| 9.584664536741213|              1|
|       1|       2.0|        17.0| 11.76470588235294|              1|
|       1|       3.2|        16.1|19.875776397515526|              1|
+--------+----------+------------+------------------+---------------+
only showing top 5 rows



**`F.lit()` - Introduciendo un Valor Literal**

¿Qué pasa si quieres añadir una columna donde *todas* las filas tengan el mismo valor (ej. "Hola", o el número 100)?

No puedes solo escribir `"Hola"`, porque Spark pensará que es el *nombre* de una columna. Debes usar `F.lit()` (literal) para decirle "este es un valor constante, no el nombre de una columna".

In [20]:
# Añadimos una columna 'fuente' con el valor constante 'taxis_ny'
df_con_literal = df_spark.withColumn("fuente", F.lit("taxis_ny"))

df_con_literal.select("passenger_count", "fuente").show(5)

+---------------+--------+
|passenger_count|  fuente|
+---------------+--------+
|              1|taxis_ny|
|              1|taxis_ny|
|              1|taxis_ny|
|              1|taxis_ny|
|              1|taxis_ny|
+---------------+--------+
only showing top 5 rows



**`withColumnRenamed()` y `drop()` - Limpieza**

Funciones básicas de limpieza: renombrar y eliminar.

In [21]:
df_spark.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)



In [22]:
# Renombremos 'passenger_count' a 'pasajeros' y eliminemos 'store_and_fwd_flag'
df_limpio = df_spark.withColumnRenamed("passenger_count", "pasajeros") \
                    .drop("store_and_fwd_flag")

print("Columnas después de renombrar y eliminar:")
df_limpio.printSchema() # .printSchema() nos muestra la estructura

Columnas después de renombrar y eliminar:
root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- pasajeros: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)



### Parte 3: El Poder de `F` (Lógica, Fechas y Nulos)

Aquí es donde `pyspark.sql.functions` (nuestro `F`) realmente muestra su poder.

**`F.when()` - El `CASE WHEN` de SQL**

Esta es *extremadamente* poderosa. Te permite crear lógica condicional (if/then/else) para una nueva columna.

In [23]:
# Creemos una columna "tipo_viaje" basada en la distancia
df_con_when = df_spark.withColumn("tipo_viaje",
    F.when(F.col("trip_distance") > 20, F.lit("Largo (Aero)"))
        .when(F.col("trip_distance") > 10, F.lit("Largo (No Aero)"))
     .when(F.col("trip_distance") > 5, F.lit("Medio"))
     .otherwise(F.lit("Corto"))
)

print("DataFrame con columna condicional 'tipo_viaje':")
df_con_when.select("trip_distance", "tipo_viaje").show(20)

DataFrame con columna condicional 'tipo_viaje':
+-------------+---------------+
|trip_distance|     tipo_viaje|
+-------------+---------------+
|         1.72|          Corto|
|          1.8|          Corto|
|          4.7|          Corto|
|          1.4|          Corto|
|          0.8|          Corto|
|          4.7|          Corto|
|        10.82|Largo (No Aero)|
|          3.0|          Corto|
|         5.44|          Medio|
|         0.04|          Corto|
|         0.75|          Corto|
|          1.2|          Corto|
|          8.2|          Medio|
|          0.4|          Corto|
|          0.8|          Corto|
|          5.0|          Corto|
|          1.5|          Corto|
|          0.0|          Corto|
|          1.5|          Corto|
|         2.57|          Corto|
+-------------+---------------+
only showing top 20 rows



**Manejo de Nulos ( `isNull`, `isNotNull`, `fillna` )**

Los datos del mundo real están sucios. `passenger_count` puede tener nulos.

In [24]:
# Contemos cuántos nulos hay
conteo_nulos = df_spark.filter(F.col("passenger_count").isNull()).count()
print(f"Viajes con 'passenger_count' nulo: {conteo_nulos}")

# Usemos fillna() para reemplazar nulos con un valor (ej. 1 pasajero por defecto)
# .fillna() es un método del DataFrame, no de F.
df_sin_nulos = df_spark.fillna(1, subset=["passenger_count"])

Viajes con 'passenger_count' nulo: 140162


In [25]:
df_sin_nulos = df_spark.fillna(1, subset=["passenger_count"])
conteo_nulos = df_sin_nulos.filter(F.col("passenger_count").isNull()).count()
print(f"Viajes con 'passenger_count' nulo en df_sin_nulos: {conteo_nulos}")

Viajes con 'passenger_count' nulo en df_sin_nulos: 0


**Funciones de Fecha/Hora (`year`, `month`, `dayofweek`)**

Nuestras columnas `tpep_pickup_datetime` son *timestamps*. Podemos extraer sus componentes.

In [26]:
# Extraigamos el año, mes, día y hora de recogida
df_con_fechas = df_spark.withColumn("año", F.year(F.col("tpep_pickup_datetime"))) \
                        .withColumn("mes", F.month(F.col("tpep_pickup_datetime"))) \
                        .withColumn("dia_semana", F.dayofweek(F.col("tpep_pickup_datetime"))) \
                        .withColumn("hora", F.hour(F.col("tpep_pickup_datetime")))

print("DataFrame con componentes de fecha/hora extraídos:")
df_con_fechas.select("tpep_pickup_datetime", "año", "mes", "dia_semana", "hora").show(5)

DataFrame con componentes de fecha/hora extraídos:
+--------------------+----+---+----------+----+
|tpep_pickup_datetime| año|mes|dia_semana|hora|
+--------------------+----+---+----------+----+
| 2024-01-01 00:57:55|2024|  1|         2|   0|
| 2024-01-01 00:03:00|2024|  1|         2|   0|
| 2024-01-01 00:17:06|2024|  1|         2|   0|
| 2024-01-01 00:36:38|2024|  1|         2|   0|
| 2024-01-01 00:46:51|2024|  1|         2|   0|
+--------------------+----+---+----------+----+
only showing top 5 rows



**Cálculo de Duración**

Podemos hacer aritmética con fechas. Calculemos la duración del viaje en segundos.

In [27]:
# Restar timestamps nos da un "Intervalo". 
# Necesitamos convertirlo a segundos usando .cast("long")
df_con_duracion = df_spark.withColumn("duracion_segundos",
    (F.col("tpep_dropoff_datetime").cast("long") - F.col("tpep_pickup_datetime").cast("long"))
)

df_con_duracion.select("duracion_segundos").show(5)

AnalysisException: [DATATYPE_MISMATCH.CAST_WITHOUT_SUGGESTION] Cannot resolve "CAST(tpep_dropoff_datetime AS BIGINT)" due to data type mismatch: cannot cast "TIMESTAMP_NTZ" to "BIGINT".;
'Project [VendorID#1174, tpep_pickup_datetime#1175, tpep_dropoff_datetime#1176, passenger_count#1177L, trip_distance#1178, RatecodeID#1179L, store_and_fwd_flag#1180, PULocationID#1181, DOLocationID#1182, payment_type#1183L, fare_amount#1184, extra#1185, mta_tax#1186, tip_amount#1187, tolls_amount#1188, improvement_surcharge#1189, total_amount#1190, congestion_surcharge#1191, Airport_fee#1192, (cast(tpep_dropoff_datetime#1176 as bigint) - cast(tpep_pickup_datetime#1175 as bigint)) AS duracion_segundos#1946]
+- Relation [VendorID#1174,tpep_pickup_datetime#1175,tpep_dropoff_datetime#1176,passenger_count#1177L,trip_distance#1178,RatecodeID#1179L,store_and_fwd_flag#1180,PULocationID#1181,DOLocationID#1182,payment_type#1183L,fare_amount#1184,extra#1185,mta_tax#1186,tip_amount#1187,tolls_amount#1188,improveme

En la Celda anterior, te encontraste con un `AnalysisException`. ¡Felicidades! Este es tu primer error de *tipos de datos* en Spark.

**El Error:** `AnalysisException: [DATATYPE_MISMATCH.CAST_WITHOUT_SUGGESTION] ... cannot cast "TIMESTAMP_NTZ" to "BIGINT"`

**Análisis del Error:**
* El error nos dice que Spark no sabe cómo convertir un `TIMESTAMP_NTZ` (un *timestamp* o marca de tiempo, sin zona horaria) directamente a un `BIGINT` (un número entero largo, como `long`).
* Tu código `F.col(...).cast("long")` es lo que generó este error.
* En versiones antiguas de Spark, esto a veces funcionaba, ya que `cast("long")` era un atajo para "dame los segundos *epoch*". En las versiones modernas, Spark es mucho más estricto con los tipos de datos para evitar ambigüedades. No sabe si quieres los segundos, los milisegundos o los microsegundos.

**La Solución:**
Para obtener la duración en segundos, primero debemos convertir el *timestamp* a un tipo numérico que represente los segundos *epoch*. El tipo de dato correcto para esto es **`double`** (un número de punto flotante de alta precisión).

Aquí está la celda corregida:

In [28]:
df_time_review = df_spark.withColumn("New DropOff",F.unix_timestamp(F.col("tpep_dropoff_datetime")))
df_time_review.select("New DropOff","tpep_dropoff_datetime").show(5)

+-----------+---------------------+
|New DropOff|tpep_dropoff_datetime|
+-----------+---------------------+
| 1704071863|  2024-01-01 01:17:43|
| 1704067776|  2024-01-01 00:09:36|
| 1704069301|  2024-01-01 00:35:01|
| 1704069896|  2024-01-01 00:44:56|
| 1704070377|  2024-01-01 00:52:57|
+-----------+---------------------+
only showing top 5 rows



In [29]:
df_time_review.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)
 |-- New DropOff: long (nullable = true)



In [30]:
# Celda 13 (Corregida): Cálculo de Duración
#
# Corregimos el error anterior. En lugar de .cast(), usamos
# la función explícita F.unix_timestamp() para convertir
# el timestamp en un número (segundos epoch).

df_con_duracion = df_spark.withColumn("duracion_segundos",
    (F.unix_timestamp(F.col("tpep_dropoff_datetime")) - F.unix_timestamp(F.col("tpep_pickup_datetime")))
)

# Ahora podemos hacer aritmética simple
df_con_duracion = df_con_duracion.withColumn("duracion_minutos",
    (F.col("duracion_segundos") / 60).cast("double") # Hacemos cast del *resultado*
)

print("Cálculo de duración exitoso:")
df_con_duracion.select("tpep_pickup_datetime", "tpep_dropoff_datetime", "duracion_segundos", "duracion_minutos").show(5)

# Limpiemos los viajes con duración negativa (errores de datos)
print(f"Viajes con duración negativa o cero: {df_con_duracion.filter(F.col('duracion_segundos') <= 0).count()}")

Cálculo de duración exitoso:
+--------------------+---------------------+-----------------+------------------+
|tpep_pickup_datetime|tpep_dropoff_datetime|duracion_segundos|  duracion_minutos|
+--------------------+---------------------+-----------------+------------------+
| 2024-01-01 00:57:55|  2024-01-01 01:17:43|             1188|              19.8|
| 2024-01-01 00:03:00|  2024-01-01 00:09:36|              396|               6.6|
| 2024-01-01 00:17:06|  2024-01-01 00:35:01|             1075|17.916666666666668|
| 2024-01-01 00:36:38|  2024-01-01 00:44:56|              498|               8.3|
| 2024-01-01 00:46:51|  2024-01-01 00:52:57|              366|               6.1|
+--------------------+---------------------+-----------------+------------------+
only showing top 5 rows

Viajes con duración negativa o cero: 870


**Alternativa - `F.datediff()`**

Si solo te importara la diferencia en *días* (no segundos), podrías usar `datediff`. Nota que esta función opera sobre *fechas* (`date`), no sobre *marcas de tiempo* (`timestamp`), así que primero debemos hacer un `cast("date")`.

In [31]:
df_dias = df_spark.withColumn("dias_de_viaje",
    F.datediff(
        F.col("tpep_dropoff_datetime").cast("date"),
        F.col("tpep_pickup_datetime").cast("date")
    )
)

print("\nDiferencia en DÍAS:")
df_dias.select("tpep_pickup_datetime", "tpep_dropoff_datetime", "dias_de_viaje").show(5)


Diferencia en DÍAS:
+--------------------+---------------------+-------------+
|tpep_pickup_datetime|tpep_dropoff_datetime|dias_de_viaje|
+--------------------+---------------------+-------------+
| 2024-01-01 00:57:55|  2024-01-01 01:17:43|            0|
| 2024-01-01 00:03:00|  2024-01-01 00:09:36|            0|
| 2024-01-01 00:17:06|  2024-01-01 00:35:01|            0|
| 2024-01-01 00:36:38|  2024-01-01 00:44:56|            0|
| 2024-01-01 00:46:51|  2024-01-01 00:52:57|            0|
+--------------------+---------------------+-------------+
only showing top 5 rows



### Parte 4: Agregaciones (El `groupBy` y `agg`)

Aquí volvemos al `groupBy`, pero con más funciones.

**`groupBy()` con Múltiples Agregaciones**

En lugar de solo `avg`, podemos pedirle a Spark que calcule todo a la vez: `avg`, `sum`, `max`, `min`, `count`, `countDistinct` (conteo de valores únicos).

In [34]:
# Agrupemos por 'PULocationID' (Zona de recogida)
# Calculemos múltiples estadísticas sobre esos viajes
df_agregado = df_spark.groupBy("PULocationID").agg(
    F.count("*").alias("conteo_viajes"),
    F.avg("total_amount").alias("promedio_total"),
    F.sum("total_amount").alias("suma_total"),
    F.max("trip_distance").alias("distancia_maxima"),
    F.countDistinct("payment_type").alias("tipos_pago_unicos")
)

print("Agregación completa por Zona de Recogida:")
df_agregado.orderBy(F.col("conteo_viajes").desc()).show(10) # Ordenamos por los más populares

Agregación completa por Zona de Recogida:
+------------+-------------+------------------+--------------------+----------------+-----------------+
|PULocationID|conteo_viajes|    promedio_total|          suma_total|distancia_maxima|tipos_pago_unicos|
+------------+-------------+------------------+--------------------+----------------+-----------------+
|         132|       145240| 76.57620855134614|1.1121928529997513E7|        10879.28|                5|
|         161|       143471|23.482411149291384|   3369045.009999984|        38202.66|                5|
|         237|       142708|19.453676878661152|  2776195.3199999756|           971.8|                5|
|         236|       136465| 20.00189477155298|  2729558.5699999775|           58.81|                5|
|         162|       106717| 22.88040002998577|   2441727.649999991|           71.18|                5|
|         230|       106324| 26.26924946390265|  2793051.6799999853|            80.0|                5|
|         186|       1

**Agregación Global (Sin `groupBy`)**

¿Qué pasa si quieres el promedio de `total_amount` de *todo* el DataFrame? No necesitas un `groupBy`.

In [35]:
# Usamos .agg() directamente sobre el DataFrame
df_stats_globales = df_spark.agg(
    F.avg("total_amount").alias("promedio_total_global"),
    F.sum("fare_amount").alias("total_tarifas_global"),
    F.max("trip_distance").alias("viaje_mas_largo")
)

print("Estadísticas globales de TODO el dataset:")
df_stats_globales.show()

Estadísticas globales de TODO el dataset:
+---------------------+--------------------+---------------+
|promedio_total_global|total_tarifas_global|viaje_mas_largo|
+---------------------+--------------------+---------------+
|   26.801504770952707|5.3882224760004714E7|       312722.3|
+---------------------+--------------------+---------------+



### Parte 5: Uniendo DataFrames (El `join`)

El `join` es fundamental. Vamos a crear un DataFrame "diccionario" para nuestros `payment_type` y a unirlos.

**Creando un DataFrame "Diccionario"**

Crearemos un pequeño DataFrame de Spark desde cero para mapear los IDs de pago.

In [36]:
df_spark.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       2| 2024-01-01 00:57:55|  2024-01-01 01:17:43|              1|         1.72|         1|                 N|         186|          79|           2|       17.7|  1.0|    0.5|       0.

In [37]:
# 1. Definimos los datos y el esquema (incluyendo el '0')
# (Una búsqueda rápida en Google sobre el dataset nos dice qué son 0, 5, 6)
datos_pago = [(0, "Desconocido"), # ¡Lo encontramos gracias al groupBy!
              (1, "Tarjeta de Crédito"),
              (2, "Efectivo"),
              (3, "Sin Cargo"),
              (4, "Disputa"),
              (5, "Vacío"),
              (6, "Nulo")]
esquema_pago = ["payment_id", "nombre_pago"]

# 2. Creamos el DataFrame
df_diccionario_pagos = spark.createDataFrame(datos_pago, schema=esquema_pago)

print("Nuestro DataFrame diccionario COMPLETO:")
df_diccionario_pagos.show()

Nuestro DataFrame diccionario COMPLETO:
+----------+------------------+
|payment_id|       nombre_pago|
+----------+------------------+
|         0|       Desconocido|
|         1|Tarjeta de Crédito|
|         2|          Efectivo|
|         3|         Sin Cargo|
|         4|           Disputa|
|         5|             Vacío|
|         6|              Nulo|
+----------+------------------+



**`join()` - Uniendo los DataFrames**

Ahora, unamos nuestro `df_spark` principal con nuestro `df_diccionario_pagos`.

In [38]:
# La sintaxis del join es:
# df_izquierdo.join(df_derecho, <condición_del_join>, <tipo_de_join>)
df_unido = df_spark.join(
    df_diccionario_pagos,
    df_spark.payment_type == df_diccionario_pagos.payment_id, # La condición
    "left_outer" # Tipo de join (left, inner, right, full_outer)
)

print("DataFrame unido con nombres de pago:")
df_unido.select("passenger_count", "total_amount", "nombre_pago").show(10)

DataFrame unido con nombres de pago:
+---------------+------------+------------------+
|passenger_count|total_amount|       nombre_pago|
+---------------+------------+------------------+
|              1|       18.75|Tarjeta de Crédito|
|              1|        31.3|Tarjeta de Crédito|
|              1|        17.0|Tarjeta de Crédito|
|              1|        16.1|Tarjeta de Crédito|
|              1|        41.5|Tarjeta de Crédito|
|              2|       64.95|Tarjeta de Crédito|
|              2|        12.9|Tarjeta de Crédito|
|              1|        22.7|          Efectivo|
|              0|        30.4|          Efectivo|
|              1|        36.0|          Efectivo|
+---------------+------------+------------------+
only showing top 10 rows



### Parte 6: El Benchmark (¡Ahora sí!)

Ahora repetimos un ejercicio anterior, pero con todo lo que sabemos.

**El Benchmark (Desbalanceado vs. Balanceado)**

Repetimos el experimento de la "Clase 2", pero ahora entendemos cada función (`filter`, `withColumn`, `groupBy`, `agg`, `orderBy`).

In [39]:
# --- 1. Definir la consulta compleja ---
def analizar_viajes_completo(dataframe):
    return dataframe.filter(
        (F.col("trip_distance") > 2) & (F.col("trip_distance") < 100)
    ).withColumn("tipo_viaje",
        F.when(F.col("trip_distance") > 20, F.lit("Largo"))
         .otherwise(F.lit("Medio"))
    ).groupBy("payment_type", "tipo_viaje").agg(
        F.avg("tip_amount").alias("propina_promedio"),
        F.count("*").alias("conteo_viajes")
    ).orderBy(F.col("conteo_viajes").desc())


# --- 2. Preparar los DataFrames (como lo hiciste) ---
df_spark.cache()
df_spark.count() # Forzar cacheo

df_reparticionado = df_spark.repartition(12)
df_reparticionado.cache()
df_reparticionado.count() # Forzar repartition y cacheo

# --- 3. Ejecutar Benchmarks ---
print("--- Ejecutando Benchmark 1 (Desbalanceado, 3 núcleos activos) ---")
%time analizar_viajes_completo(df_spark).show()

print("\n--- Ejecutando Benchmark 2 (Balanceado, 12 núcleos activos) ---")
%time analizar_viajes_completo(df_reparticionado).show()

# Liberamos memoria
df_spark.unpersist()
df_reparticionado.unpersist()

--- Ejecutando Benchmark 1 (Desbalanceado, 3 núcleos activos) ---
+------------+----------+--------------------+-------------+
|payment_type|tipo_viaje|    propina_promedio|conteo_viajes|
+------------+----------+--------------------+-------------+
|           1|     Medio|   6.008564521889089|       938015|
|           2|     Medio|0.002486501326988...|       163905|
|           0|     Medio|   2.234879677758519|        67527|
|           1|     Largo|  15.338880528457176|        22859|
|           4|     Medio| 0.06863114231014676|        15670|
|           2|     Largo|0.032797917470111834|         5186|
|           3|     Medio|0.028834973166368513|         4472|
|           4|     Largo|  0.3381708945260347|          749|
|           0|     Largo|   10.91443762781186|          489|
|           3|     Largo| 0.13790960451977402|          177|
+------------+----------+--------------------+-------------+

CPU times: user 1.8 ms, sys: 957 μs, total: 2.76 ms
Wall time: 514 ms

--- Ejec

DataFrame[VendorID: int, tpep_pickup_datetime: timestamp_ntz, tpep_dropoff_datetime: timestamp_ntz, passenger_count: bigint, trip_distance: double, RatecodeID: bigint, store_and_fwd_flag: string, PULocationID: int, DOLocationID: int, payment_type: bigint, fare_amount: double, extra: double, mta_tax: double, tip_amount: double, tolls_amount: double, improvement_surcharge: double, total_amount: double, congestion_surcharge: double, Airport_fee: double]

¡Aquí es donde todo el trabajo conceptual da sus frutos! Los resultados de tu benchmark son la prueba perfecta de por qué la optimización importa.

**Tus Resultados:**
* **Benchmark 1 (Desbalanceado):** `Wall time: 542 ms`
* **Benchmark 2 (Balanceado):** `Wall time: 303 ms`

**Análisis de Rendimiento:**
* Al ejecutar `df_spark.repartition(12)`, hiciste que la consulta fuera **¡un 79% más rápida!** (`542 / 303 = 1.79`).
* **¿Por qué?**
    * En el Benchmark 1, las primeras etapas (el `filter` y `withColumn`) se ejecutaron en solo **3 núcleos**, creando un "cuello de botella". Los otros 9 núcleos estaban esperando.
    * En el Benchmark 2, esas mismas etapas se ejecutaron en **12 núcleos en paralelo**.
* **Driver vs. Executors:**
    * Fíjate en el `CPU times` (tiempo del Driver): `3.37 ms` vs `6.86 ms`.
    * Esto muestra que el Driver (tu notebook) tuvo que "trabajar" un poquito más en el Benchmark 2. ¿Por qué? ¡Porque tuvo que coordinar a 12 trabajadores en lugar de a 3!
    * Pero el `Wall time` (tiempo total de reloj) es lo que importa, y se redujo drásticamente.

**¡Felicidades!** Has demostrado empíricamente cómo una optimización de distribución de datos impacta directamente el rendimiento.

Hemos cubierto:
* **Selección:** `select`, `selectExpr`
* **Filtrado:** `filter`, `where`, `like`, `isin`
* **Manipulación:** `withColumn`, `withColumnRenamed`, `drop`
* **Funciones `F`:** `lit`, `when`, `otherwise`, `isNull`, `year`, `month`, `dayofweek`, `hour`, `cast`
* **Manejo de Nulos:** `fillna`
* **Agregaciones:** `groupBy`, `agg`, `avg`, `sum`, `max`, `count`, `countDistinct`
* **Uniones:** `join`, `createDataFrame`
* **Acciones:** `show`, `count`, `printSchema` (y las de la clase pasada: `collect`, `write`, `toPandas`)


Con esto conformamos un "Cookbook" funcional para empezar a trabajar.

### Parte 6: El Siguiente Nivel - Spark SQL y UDFs

Has dominado la "API de DataFrame" (el estilo Python). Pero Spark tiene otra cara: **Spark SQL**.

Además, ¿qué pasa si la función que quieres no existe en `F`? Creas la tuya: una **UDF** (User Defined Function).

**`createOrReplaceTempView()` - Hablando SQL**

Podemos tomar *cualquier* DataFrame (como nuestro `df_unido_completo`) y registrarlo como una "tabla" temporal de SQL. Una vez hecho, ¡puedes usar sintaxis SQL pura!

In [40]:
# 1. Registramos nuestro DataFrame como una tabla SQL temporal
df_unido.createOrReplaceTempView("taxis")

# 2. ¡Escribimos SQL!
# Esto hace LO MISMO que nuestro "analizar_viajes_completo"
df_sql = spark.sql("""
    SELECT 
        payment_type, 
        nombre_pago,
        CASE 
            WHEN trip_distance > 20 THEN 'Largo'
            ELSE 'Medio'
        END AS tipo_viaje,
        AVG(tip_amount) AS propina_promedio,
        COUNT(*) AS conteo_viajes
    FROM taxis
    WHERE trip_distance > 2 AND trip_distance < 100
    GROUP BY payment_type, nombre_pago, tipo_viaje
    ORDER BY conteo_viajes DESC
""")

print("¡El mismo resultado, pero generado con 100% SQL!")
df_sql.show()

¡El mismo resultado, pero generado con 100% SQL!
+------------+------------------+----------+--------------------+-------------+
|payment_type|       nombre_pago|tipo_viaje|    propina_promedio|conteo_viajes|
+------------+------------------+----------+--------------------+-------------+
|           1|Tarjeta de Crédito|     Medio|    6.00856452188944|       938015|
|           2|          Efectivo|     Medio|0.002486501326988...|       163905|
|           0|       Desconocido|     Medio|   2.234879677758519|        67527|
|           1|Tarjeta de Crédito|     Largo|  15.338880528457112|        22859|
|           4|           Disputa|     Medio| 0.06863114231014678|        15670|
|           2|          Efectivo|     Largo|0.032797917470111834|         5186|
|           3|         Sin Cargo|     Medio| 0.02883497316636852|         4472|
|           4|           Disputa|     Largo|  0.3381708945260347|          749|
|           0|       Desconocido|     Largo|   10.91443762781186|      

**`F.udf()` - Creando tus propias funciones**

¿Qué pasa si queremos una función compleja que no existe en Spark? Por ejemplo, una que clasifique la propina en "Baja", "Media", "Alta".

**Advertencia:** Las UDF son *lentas*. Spark tiene que enviar los datos de su motor optimizado (JVM) a Python, ejecutar tu función, y devolver el resultado. **Siempre prefiere funciones `F` nativas si puedes.** Pero a veces, son inevitables.

In [42]:
from pyspark.sql.types import StringType

# 1. Definimos una función de Python normal
def clasificar_propina(propina, total):
    if total is None or total == 0:
        return "N/A"
    if propina is None:
        propina = 0
    
    pct = (propina / total) * 100
    
    if pct > 25:
        return "Alta"
    elif pct > 15:
        return "Media"
    elif pct > 0:
        return "Baja"
    else:
        return "Ninguna"

# 2. "Envolvemos" nuestra función de Python en una UDF de Spark
# Le decimos a Spark el tipo de dato que nuestra función va a devolver
clasificar_propina_udf = F.udf(clasificar_propina, StringType())

# 3. Usamos la UDF como si fuera una función F
df_con_udf = df_spark.withColumn("categoria_propina", 
    clasificar_propina_udf(F.col("tip_amount"), F.col("total_amount"))
)

print("DataFrame con nuestra UDF personalizada:")
df_con_udf.select("tip_amount", "total_amount", "categoria_propina").show(10)

# ¡Ahora podemos contar las categorías!
df_con_udf.groupBy("categoria_propina").count().orderBy(F.col("count").desc()).show()

DataFrame con nuestra UDF personalizada:
+----------+------------+-----------------+
|tip_amount|total_amount|categoria_propina|
+----------+------------+-----------------+
|       0.0|        22.7|          Ninguna|
|      3.75|       18.75|            Media|
|       3.0|        31.3|             Baja|
|       2.0|        17.0|             Baja|
|       3.2|        16.1|            Media|
|       6.9|        41.5|            Media|
|      10.0|       64.95|            Media|
|       0.0|        30.4|          Ninguna|
|       0.0|        36.0|          Ninguna|
|       0.0|         8.0|          Ninguna|
+----------+------------+-----------------+
only showing top 10 rows

+-----------------+-------+
|categoria_propina|  count|
+-----------------+-------+
|            Media|1638576|
|          Ninguna| 710378|
|             Baja| 599218|
|             Alta|  16036|
|              N/A|    416|
+-----------------+-------+

